## **1873026_임재랑**
**뉴스 데이터 수집 part**

**드라이브 연결 및 라이브러리 import**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

**줌 뉴스 데이터 수집 함수 만들기**

In [3]:

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}


def ZUM_news_scraper(maxpage, query, s_date, e_date):
    
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    
    news_df = pd.DataFrame(columns=("Title", "Datetime", "Article", "Link"))
    index = 0 
        
    page = 1
    maxpage_t =(int(maxpage)) 

    
    while page <= maxpage_t:
        
        url = 'https://search.zum.com/search.zum?method=news&option=accu&query={0}&rd=1&cluster=&startdate={1}&enddate={2}&datetype=input&scp=0&page={3}&mm=direct'.format(query, s_date, e_date, page)

        print('▶ 페이지 번호 = ', page) 
        req = requests.get(url)
        
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        
        for urls in soup.select(".report-link"):
            
            try:
                if urls["href"].startswith("https://news.zum.com"):
                        
                        newsreq = requests.get(urls["href"], headers = headers)
                        
                        bsoup = BeautifulSoup(newsreq.content, 'html.parser')

                        title = bsoup.select('h1#news_title')[0].text[53:]

                        newsdate = bsoup.select('dl.time dd')[0].get_text()[:10]

                        btext = bsoup.select('.article_body')[0].get_text().replace('\n', " ").replace('\t', " ")
                        article = btext.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")

                        newscompany = bsoup.select('a.media')[0].get_text()   
                        
                        news_df.loc[index] = [title, newsdate, article, newscompany]
                        index += 1              
                                            

            except Exception as e:
                continue
        page += 1
        
    print()
    print('Completed!🌞 ')
    
    return news_df 
    

    


**정보 입력받기**

In [13]:

query = input("◎ 검색어: ")

maxpage = input("◎ 검색 할 페이지수: ")   

s_date = input("◎ 시작 날짜(YYYYMMDD): ")

e_date = input("◎ 종료 날짜(YYYYMMDD): ")


news_df = ZUM_news_scraper(maxpage, query, s_date, e_date)




◎ 검색어: SF 영화
◎ 검색 할 페이지수: 20
◎ 시작 날짜(YYYYMMDD): 20100101
◎ 종료 날짜(YYYYMMDD): 20120601
▶ 페이지 번호 =  1
▶ 페이지 번호 =  2
▶ 페이지 번호 =  3
▶ 페이지 번호 =  4
▶ 페이지 번호 =  5
▶ 페이지 번호 =  6
▶ 페이지 번호 =  7
▶ 페이지 번호 =  8
▶ 페이지 번호 =  9
▶ 페이지 번호 =  10
▶ 페이지 번호 =  11
▶ 페이지 번호 =  12
▶ 페이지 번호 =  13
▶ 페이지 번호 =  14
▶ 페이지 번호 =  15
▶ 페이지 번호 =  16
▶ 페이지 번호 =  17
▶ 페이지 번호 =  18
▶ 페이지 번호 =  19
▶ 페이지 번호 =  20

Completed!🌞 


In [14]:
news_df.shape

(170, 4)

In [15]:
news_df.head() 

,Title,Datetime,Article,Link
0,환상적인 조개집 “실제 건축물이라고? SF영화 같다” ...,2012.01.01,조개껍데기 모양의 건축물 사진이 누리꾼들을 놀라게 하고 있다. 조개껍데기 모양...,매일경제
1,영화에 '우주'를 담으면 뜬다?..SF 영화 '흥행불패' ...,2012.04.18,[OSEN=김경주 기자] 우주를 배경으로 했던 영화들이 모두 흥행에 성공하며 ...,OSEN
2,"[기획-2] SF영화같은 의료시뮬레이션 ""교육 효과는 현실적"" ...",2012.04.09,서울의 A대학병원 분만실 앞. 한 산부인과 교수가 난처한 표정으로 서 있다....,라포르시안
3,"[영상] 뉴에프오, '바운스' 뮤비 공개 ""SF영화 같아"" ...",2011.11.24,5인조 신인 걸그룹 뉴에프오(New.F.O)의 첫 번째 디지털 싱글 타이틀곡 ...,파이낸셜뉴스
4,"2층 열차, 하늘 나는 배 … SF영화서 현실 속으로 ...",2011.09.29,...,중앙일보


**데이터 파일 드라이브에 저장**

In [17]:
file_path =  "/content/gdrive/My Drive/Colab Notebooks/웹과텍스트마이닝기말프로젝트/"

file_name = "줌_뉴스_" + query + "_" + s_date + "_" + e_date + ".csv"

file_full_path = file_path + file_name

news_df.to_csv(file_full_path,index = False, encoding="utf-8-sig")  

**불러오기 확인**

In [18]:
report = pd.read_csv(file_full_path)
report.shape

(170, 4)

In [19]:
report

,Title,Datetime,Article,Link
0,환상적인 조개집 “실제 건축물이라고? SF영화 같다” ...,2012.01.01,조개껍데기 모양의 건축물 사진이 누리꾼들을 놀라게 하고 있다. 조개껍데기 모양...,매일경제
1,영화에 '우주'를 담으면 뜬다?..SF 영화 '흥행불패' ...,2012.04.18,[OSEN=김경주 기자] 우주를 배경으로 했던 영화들이 모두 흥행에 성공하며 ...,OSEN
2,"[기획-2] SF영화같은 의료시뮬레이션 ""교육 효과는 현실적"" ...",2012.04.09,서울의 A대학병원 분만실 앞. 한 산부인과 교수가 난처한 표정으로 서 있다....,라포르시안
3,"[영상] 뉴에프오, '바운스' 뮤비 공개 ""SF영화 같아"" ...",2011.11.24,5인조 신인 걸그룹 뉴에프오(New.F.O)의 첫 번째 디지털 싱글 타이틀곡 ...,파이낸셜뉴스
4,"2층 열차, 하늘 나는 배 … SF영화서 현실 속으로 ...",2011.09.29,...,중앙일보
...,...,...,...,...
165,"'백투더퓨쳐' 타임머신, 경매 나왔다…경매가 7억 예상 ...",2011.10.19,[마이데일리 = 강지훈 기자] 세계적인 성공을 거둔 마이클 제이 폭스의 영화 ...,마이데일리
166,"이송희일, ""문제는 심형래라는 개인이 아니라…"" ...",2011.10.19,[마이데일리 = 최두선 기자] 영화감독 이송희일이 18일 오후 MBC 'PD수...,마이데일리
167,"PD수첩 영구의 몰락, 국고지원 선정 중 정관계 인사 개입의혹! ...",2011.10.19,‘PD수첩’ ‘영구의 몰락’에서는 심형래가 설립한 영구아트의 특혜 의혹을 제기...,파이낸셜뉴스
168,'PD수첩' 심형래 효과? 시청률8.1% 2배 급증! ...,2011.10.19,[마이데일리 = 최두선 기자] 개그맨 겸 영화감독 심형래의 비리사실을 파헤친 ...,마이데일리
